# Álgebra Lineal Computacional
## TP Red Neuronal Lineal

In [1]:
import numpy as np
import time
from alc import *

## 1. Lectura de datos

In [2]:
BASE_PATH = "template-alumnos/dataset/cats_and_dogs"

# --- Training dataset ---
X_cats = np.load(f'{BASE_PATH}/train/cats/efficientnet_b3_embeddings.npy')
X_dogs = np.load(f'{BASE_PATH}/train/dogs/efficientnet_b3_embeddings.npy')

X_train = np.hstack([X_cats, X_dogs])  

Y_train = np.hstack([
    np.tile([[1],[0]], (1, X_cats.shape[1])),
    np.tile([[0],[1]], (1, X_dogs.shape[1]))
])                                             

# --- Validation dataset ---
V_cats = np.load(f'{BASE_PATH}/val/cats/efficientnet_b3_embeddings.npy')
V_dogs = np.load(f'{BASE_PATH}/val/dogs/efficientnet_b3_embeddings.npy')

X_val = np.hstack([V_cats, V_dogs])
Y_val = np.hstack([
    np.tile([[1],[0]], (1, V_cats.shape[1])),
    np.tile([[0],[1]], (1, V_dogs.shape[1]))
])

In [3]:
print(f'X_train: {X_train.shape}, Y_train: {Y_train.shape}')
print(f'X_val: {X_val.shape} Y_val: {Y_val.shape}')

X_train: (1536, 2000), Y_train: (2, 2000)
X_val: (1536, 1000) Y_val: (2, 1000)


## 3. Descomposición SVD

In [7]:
S, D, V = svd_reducida(X_train, atol=1e-1)

KeyboardInterrupt: 

## 4. Descomposición QR

Los métodos `pinvGramSchmidt` y `pinvHouseHolder` son idénticos porque el cálculo de $W$ a partir de $Q, R, Y$ no depende del tipo de factorización.

Sí cabe destacar que podrían optimizarse los métodos `inversa` y `multiplicar` haciendo uso de que $R$ es matriz triangular.

In [16]:
def pinvGramSchmidt(Q, R, Y):
    """
    Recibe Q, R factores de X.T y la matriz Y.
    Calcula X⁺ = Q (R.T)⁻¹
    Calcula W = Y X⁺
    """
    R_T = transpuesta(R)
    R_inv = inversa(R_T)
    W = multiplicar(Y, multiplicar(Q, R_inv))
    return W

def pinvHouseHolder(Q, R, Y):
    """
    Recibe Q, R factores de X.T y la matriz Y.
    Calcula X⁺ = Q (R.T)⁻¹
    Calcula W = Y X⁺
    """
    R_T = transpuesta(R)
    R_inv = inversa(R_T)
    W = multiplicar(Y, multiplicar(Q, R_inv))
    return W

#### Gram Schmidt

In [ ]:
# Factorizamos
Q_gs, R_gs = QR_con_GS(transpuesta(X_train))

# Guardamos matrices
np.save('Q_gs.npy', Q_gs)
np.save('R_gs.npy', R_gs)

In [25]:
Q_gs = np.load('Q_gs.npy')
R_gs = np.load('R_gs.npy')

In [26]:
print(f'Q_gs: {Q_gs.shape}, R_gs: {R_gs.shape}')

Q_gs: (2000, 1536), R_gs: (2000, 1536)


Recortamos $R_{gs}$ para reducir su tamaño a una cuadrada (**las filas inferiores de la matriz R son nulas** como resultado de la factorización).

In [13]:
# Las filas inferiores son nulas
assert np.allclose(R_gs[1536:, :], 0, atol=1e-12)

In [27]:
# Recortamos la matriz R
R_gs = R_gs[:1536, :]
print(f'Q_gs: {Q_gs.shape}, R_gs: {R_gs.shape}')

Q_gs: (2000, 1536), R_gs: (1536, 1536)


In [ ]:
W_gs = pinvHouseHolder(Q_gs, R_gs, Y_train)
np.save('W_gs.npy', Q_gs)

In [19]:
W_gs = np.load('W_gs.npy')
W_gs.shape

(2, 1536)

#### Householder

In [ ]:
# Factorizamos
Q_hh, R_hh = QR_con_HH(transpuesta(X_train))

# Guardamos matrices
np.save('Q_hh.npy', Q_hh)
np.save('R_hh.npy', R_hh)

In [20]:
Q_hh = np.load('Q_hh.npy')
R_hh = np.load('R_hh.npy')

In [21]:
print(f'Q_hh: {Q_hh.shape}, R_hh: {R_hh.shape}')

Q_hh: (2000, 2000), R_hh: (2000, 1536)


Recortamos $Q_{gs}$ y $R_{gs}$ para reducir los tamaños (las filas inferiores de la matriz R son nulas como resultado de la factorización).

In [13]:
# Las filas inferiores son nulas
assert np.allclose(R_gs[1536:, :], 0, atol=1e-12)

In [23]:
# Recortamos las matrices
Q_hh = Q_hh[:, :1536]
R_hh = R_hh[:1536, :]
print(f'Q_hh: {Q_hh.shape}, R_hh: {R_hh.shape}')

Q_hh: (2000, 1536), R_hh: (1536, 1536)


In [ ]:
W_hh = pinvHouseHolder(Q_hh, R_hh, Y_train)
np.save('W_hh.npy', W_hh)

In [22]:
W_hh = np.load('W_hh.npy')
W_hh.shape

(2, 1536)

## 7. Síntesis final

### Metodología QR

#### Optimización
Surgió la necesidad de optimizar los algoritmos de factorización QR.

Reemplazamos la multiplicación de matrices enteras por bloques de tamaño reducido; aprovechando la lógica de filas/columnas nulas o invariantes.

Medimos la mejora con un benchmark de 30 muestras de matrices 30 x 30; para el la factorización Housholder obtuvimos tiempos entre 10 a 100 veces mejores:

| Algoritmo | Tiempo promedio | Mejor tiempo |
|:---|:---:|:---:|
| NumPy LAPACK | 0.00017 seg | 0.00013 seg |
| Gram-Schmidt naif | 0.04047 seg | 0.03340 seg |
| Gram-Schmidt optimizado | 0.01434 seg | 0.01184 seg |
| Householder naif | 1.02379 seg | 0.85465 seg |
| Householder optimizado | 0.03032 seg | 0.02463 seg |